In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [37]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [38]:
params[0]

Parameter containing:
tensor([[[[ 0.2311,  0.3148,  0.0065],
          [ 0.0175, -0.0341,  0.2625],
          [ 0.0814,  0.2260,  0.2195]]],


        [[[ 0.0652,  0.0424, -0.0160],
          [-0.3136,  0.3257,  0.1196],
          [ 0.0614,  0.0662,  0.0128]]],


        [[[-0.0403,  0.1257,  0.0668],
          [-0.1146,  0.1957,  0.1000],
          [-0.2193,  0.2989,  0.1063]]],


        [[[ 0.0546,  0.2351,  0.2594],
          [-0.2012,  0.2349,  0.1593],
          [ 0.0915, -0.2505,  0.1510]]],


        [[[ 0.2796,  0.2061, -0.1857],
          [ 0.2713, -0.1148, -0.0195],
          [-0.2146,  0.0741, -0.0729]]],


        [[[ 0.0134, -0.1331,  0.0660],
          [-0.1738,  0.1793,  0.2118],
          [-0.2973,  0.0410,  0.0574]]]], requires_grad=True)

In [39]:
net.parameters()

<generator object Module.parameters at 0x7f505c3cb048>

In [40]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0482,  0.0259,  0.1276,  0.0747, -0.0202,  0.0132,  0.0190, -0.0248,
         -0.1053,  0.0035]], grad_fn=<AddmmBackward>)


In [41]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [42]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7959, grad_fn=<MseLossBackward>)


In [43]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [44]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0030, -0.0021, -0.0035, -0.0095,  0.0061, -0.0096])


In [45]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

In [46]:
optimizer.zero_grad()
output = net(input)

In [47]:
loss = criterion(output, target)
loss.backward()
optimizer.step()
